# Importing Libraries

In [1]:
import sklearn
import os
import pandas as pd
import numpy as np

Cloning git repository related to this project. Crawled data and Crawler code are available in git repository.

In [2]:
!git clone https://github.com/tandalalam/saadi-retreival.git

Cloning into 'saadi-retreival'...
remote: Enumerating objects: 585, done.
remote: Counting objects: 100% (585/585), done.
remote: Compressing objects: 100% (577/577), done.
remote: Total 585 (delta 16), reused 563 (delta 5), pack-reused 0
Receiving objects: 100% (585/585), 356.73 KiB | 11.89 MiB/s, done.
Resolving deltas: 100% (16/16), done.


In [3]:
os.chdir('saadi-retreival')

Reading Boostan from file:

In [4]:
boostan_data = list()
for chapter_name in os.listdir('boostan'):
    for file_name in os.listdir(f'boostan/{chapter_name}'):
        file = open(f'boostan/{chapter_name}/{file_name}')
        poem = file.read()
        for hem in [x.strip() for x in poem.split('\n\n')]:
            if hem:
                boostan_data.append([hem, int(chapter_name.split('bab')[1]), int(file_name.split('sh')[1].split('.')[0])])

In [5]:
boostan_data = pd.DataFrame(boostan_data)

In [6]:
boostan_data.columns = ['poem', 'chapter', 'section']

In [7]:
boostan_data.sort_values(by=['chapter', 'section'], ignore_index=True, inplace=True)

In [8]:
boostan_data

,poem,chapter,section
0,به نام خداوندِ جان‌آفرین\nحکیمِ سخن‌درزبان‌آفرین,0,1
1,خداوند بخشندهٔ دستگیر\nکریم خطابخش پوزش‌پذیر,0,1
2,عزیزی که هر کز درش سر بتافت\nبه هر در که شد هی...,0,1
3,سر پادشاهان گردن‌فراز\nبه درگاه او بر زمین نیاز,0,1
4,نه گردن‌کشان را بگیرد به فور\nنه عذرآوران را ب...,0,1
...,...,...,...
4059,به کردار بدشان مقید نکرد\nبضاعات مزجاتشان رد نکرد,10,4
4060,ز لطفت همین چشم داریم نیز\nبر این بی‌بضاعت ببخ...,10,4
4061,کس از من سیه نامه تر دیده نیست\nکه هیچم فعال پ...,10,4
4062,جز این کاعتمادم به یاری تست\nامیدم به آمرزگاری...,10,4


Reading Golestan from file:

In [9]:
import re

We will split Golestan's NASR based on their punctiuation:

In [10]:
golestan_data = list()
for chapter_name in os.listdir('golestan'):
    for file_name in os.listdir(f'golestan/{chapter_name}'):
        file = open(f'golestan/{chapter_name}/{file_name}')
        for hem in [x.strip() for x in re.split('\n\n|\.|\?|!', file.read())]:
            if len(hem.split()) < 3:
              continue
            if hem:
                if file_name.startswith('d'):
                    golestan_data.append([hem, 0, 1])
                else:
                    golestan_data.append([hem, int(chapter_name.split('bab')[1]), int(file_name.split('sh')[1].split('.')[0])])

In [11]:
golestan_data = pd.DataFrame(golestan_data)

In [12]:
golestan_data.columns = ['poem', 'chapter', 'section']

In [13]:
golestan_data.sort_values(by=['chapter', 'section'], ignore_index=True, inplace=True)

In [14]:
golestan_data

,poem,chapter,section
0,بِسم اللهِ الرَّحمنِ الرَّحیم,0,1
1,منّت خدای را عز و جل که طاعتش موجب قربت است و ...,0,1
2,هر نفسی که فرو می رود ممدّ حیات است و چون بر م...,0,1
3,پس در هر نفسی دو نعمت موجود است و بر هر نعمتی ...,0,1
4,از دست و زبان که بر آید\nکز عهدهٔ شکرش به در آید,0,1
...,...,...,...
2497,غالب گفتار سعدی طرب انگیز است و طیبت آمیز و کو...,8,109
2498,ما نصیحت به جای خود کردیم\nروزگاری در این به س...,8,109
2499,گر نیاید به گوش رغبت کس\nبر رسولان پیام باشد و بس,8,109
2500,یا ناظراً فیه سَل باللهِ مرحمةً\nعلی المصنفِ و...,8,109


Concatanating data from Boostan and Golestan:

In [15]:
all_data = pd.concat([boostan_data, golestan_data], ignore_index=True)['poem']

In [16]:
!pip3 install hazm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 316 kB 28.3 MB/s 
     |████████████████████████████████| 1.4 MB 59.0 MB/s 
     |████████████████████████████████| 233 kB 65.8 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394487 sha256=551b018629b62d572aee94453ede4c7f3cf6d38894a311a12c2d68230f290fcf
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=153385 sha256=17315835737ce9881d659dc22ce3fceb1700c30338e3ec5a0bd4d4d22990ce78
  Stored in directory: /root/.cache/pip/wheels/ab/b2/5b/0fe4b8f5c0e65341e8ea7bb3f4a6ebabfe8b1ac31322392dbf
Successfully built nltk libwapiti
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [17]:
from __future__ import unicode_literals
from hazm import *
import tqdm
import string

Importing stopwords and words that should be replaced:  
We have use stop-words for persian poems from [here](https://github.com/amnghd/Persian_poems_corpus). And word that should replaced are written by ourselves.

In [18]:
stopwords = []
replace_dict = {}
punctuations = '\.:!،؛؟»\]\)\}«\[\(\{' + string.punctuation

with open('stopwords.txt') as f:
    var = f.readline()
    while var:
        stopwords.append(var.strip())
        var = f.readline()

with open('replace.txt') as f:
    line = f.readline()
    while line:
        key, value = line.split('-')
        key, value = key.strip(), value.strip()
        replace_dict[f'{key}'] = f'{value}'
        line = f.readline()

# Preprocessing Data

In [19]:
normalizer = Normalizer()
stemmer = Stemmer()
lemmatizer = Lemmatizer()

def replace_function(string):
    if string in replace_dict:
      return replace_dict[string]
    return string

def sent_pre_process(sentence, normalize=True, remove_stopwords=False, stemme=False, lemmatize=True, replace=True, remove_punctuations=True, is_first=True):

    # replace some charachters
    replace_char = {'هٔ': 'ه',
                    'ۀ' : 'ه',
                    'ه‌ی' : 'ه'}
    
    if remove_punctuations:
      for char in punctuations:
        replace_char[char] = " "

    for key, value in replace_char.items():
        sentence = sentence.replace(key, value)

    if normalize:
        sentence = normalizer.normalize(sentence)
    if stemme:
        sentence = stemmer.stemme(sentence)
    if lemmatize:
        sentence = lemmatizer.lemmatize(sentence)
    

    tokens = word_tokenize(sentence)
    
    if replace:
        tokens = [replace_function(token) for token in tokens]
    if remove_stopwords:
        tokens = [token for token in tokens if token not in stopwords]
    
    if is_first:
        return sent_pre_process(" ".join(tokens), normalize, remove_stopwords, stemme, lemmatize, replace, remove_punctuations, False)
    
    return tokens

def pre_process(sentences, normalize=True, remove_stopwords=False, stemme=False, lemmatize=True, replace=True, remove_punctuations=True, is_first=True, MIN_COUNT=3, put_index=False):
    if put_index:
        processed = [(sent_pre_process(sent, normalize, remove_stopwords, stemme, lemmatize, replace, remove_punctuations, is_first), i) for i, sent in enumerate(sentences)]
        return [t for t in processed if len(list(t[0])) > MIN_COUNT]
    processed = [sent_pre_process(sent, normalize, remove_stopwords, stemme, lemmatize, replace, remove_punctuations, is_first) for sent in sentences]
    return [t for t in processed if len(list(t)) > MIN_COUNT]

# Boolean Retreival

For convinience, we use CountVectorizer and then convert its values from number to boolean to get boolean retreival matrix:

In [20]:
from sklearn.feature_extraction.text import CountVectorizer
tf = CountVectorizer(analyzer= lambda x: sent_pre_process(x, remove_stopwords=True))
tf_data = tf.fit_transform(all_data)
tf_data = tf_data.toarray().astype(bool)

In [21]:
tf.transform(['ما ز یاران چشم یاری']).toarray()

array([[0, 0, 0, ..., 0, 0, 0]])

In [22]:
tf_data

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [23]:
def boolean_retrieval(str, k=10):
  query_vec = tf.transform([str]).toarray()[0]
  scores = np.dot(tf_data, query_vec)
  args = np.argsort(scores)[::-1]
  k_args = args[:k]
  return np.concatenate([[scores[k_args]], [all_data[k_args]]]).T

In [24]:
boolean_retrieval('به نام خداوند جان و خرد کز این برتر اندیشه بر نگذرد')

array([[2, 'اگر باد سرد نفس نگذرد\nتف معده جان در خروش آورد'],
       [1,
        'به نام خداوندِ جان\u200cآفرین\nحکیمِ سخن\u200cدرزبان\u200cآفرین'],
       [1,
        'در این کشور اندیشه کردم بسی\nپریشان\u200cتر از خود ندیدم کسی'],
       [1, 'عجب آن که غراب از مجاورت طوطی هم به جان آمده بود و ملول شده'],
       [1, 'هر آن کس که جور بزرگان نبرد\nنسوزد دلش بر ضعیفان خرد'],
       [1,
        'که درمانده\u200cام دست گیر ای صنم\nبه جان آمدم رحم کن بر تنم'],
       [1,
        'یکی را زنی صاحب جمال جوان درگذشت و مادرزن فرتوت به علت کابین در خانه متمکن بماند و مرد از محاورت او به جان رنجیدی و از مجاورت او چاره ندیدی تا گروهی آشنایان به پرسیدن آمدندش'],
       [1,
        'چو الب ارسلان جان به جان\u200cبخش داد\nپسر تاج شاهی به سر برنهاد'],
       [1, 'ز دوران ملک پدر یاد کن\nدل از بند اندیشه آزاد کن'],
       [1, 'وگر راست گفت ای خداوند پاک\nمرا توبه ده تا نگردم هلاک']],
      dtype=object)

We can train a retreival system that returns the whole related poem (file) instead of only one MESRAA or BEIT just as easily:

In [57]:
# setting book column for both golestan and boostan:
golestan_data['book'] = 'Golestan'
boostan_data['book'] = 'Boostan'

In [75]:
df = all_data = pd.concat([boostan_data, golestan_data], ignore_index=True)
file_based_df = df.groupby(['book', 'chapter', 'section']).apply(lambda f: ' '.join(f['poem'].tolist())).reset_index()
file_based_df.columns = ['book', 'chapter', 'section', 'poem']
file_based_df.head()

,book,chapter,section,poem
0,Boostan,0,1,به نام خداوندِ جان‌آفرین\nحکیمِ سخن‌درزبان‌آفر...
1,Boostan,0,2,کریم السجایا جمیل الشیم\nنبی البرایا شفیع الام...
2,Boostan,0,3,در اقصای عالم بگشتم بسی\nبه سر بردم ایام با هر...
3,Boostan,0,4,مرا طبع از این نوع خواهان نبود\nسر مدحت پادشاه...
4,Boostan,0,5,اتابک محمد شه نیکبخت\nخداوند تاج و خداوند تخت ...


In [89]:
file_base_tf = CountVectorizer(analyzer= lambda x: sent_pre_process(x, remove_stopwords=True))
file_base_tf_data = file_base_tf.fit_transform(file_based_df.poem)
file_base_tf_data = file_base_tf_data.toarray().astype(bool)

def file_base_boolean_retrieval(str, k=10):
  query_vec = file_base_tf.transform([str]).toarray()[0]
  scores = np.dot(file_base_tf_data, query_vec)
  args = np.argsort(scores)[::-1]
  k_args = args[:k]
  return np.concatenate([[scores[k_args]], [file_based_df.poem[k_args]]]).T

In [90]:
file_base_boolean_retrieval('به نام خداوند جان و خرد کز این برتر اندیشه بر نگذرد', k=1)

array([[4,
        'بِسم اللهِ الرَّحمنِ الرَّحیم منّت خدای را عز و جل که طاعتش موجب قربت است و به شکر اندرش مزید نعمت هر نفسی که فرو می رود ممدّ حیات است و چون بر می آید مفرّح ذات پس در هر نفسی دو نعمت موجود است و بر هر نعمتی شکری واجب از دست و زبان که بر آید\nکز عهدهٔ شکرش به در آید اِعملوا آلَ داودَ شکراً وَ قلیلٌ مِن عبادیَ الشکور بنده همان به که ز تقصیر خویش\nعذر به درگاه خدای آورد ور نه سزاوار خداوندیش\nکس نتواند که به جای آورد باران رحمت بی حسابش همه را رسیده و خوان نعمت بی دریغش همه جا کشیده پردهٔ ناموس بندگان به گناه فاحش ندرد و وظیفهٔ روزی به خطای منکر نبرد ای کریمی که از خزانهٔ غیب\nگبر و ترسا وظیفه خور داری دوستان را کجا کنی محروم\nتو که با دشمن این نظر داری فرّاش باد صبا را گفته تا فرش زمرّدی بگسترد و دایهٔ ابر بهاری را فرموده تا بنات نبات در مهد زمین بپرورد درختان را به خلعت نوروزی قبای سبز ورق در بر گرفته و اطفال شاخ را به قدوم موسم ربیع کلاه شکوفه بر سر نهاده عصاره نالی به قدرت او شهد فایق شده و تخم خرمایی به تربیتش نخل باسق گشته ابر و باد و مه و خورشید و فلک در کارند\n

And we can use this to create our file-based retreival system for all our models. But in this notebook, our main retreival systems are retreiving BEITs.

# TF-IDF Retreival

Defining cosine function to compute cosine of angle between two vectors in space:


In [25]:
def get_cosine(v1,v2):
    return np.inner(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

get_most_similars returns k most similar vectos to query_vector based on similarity_func:


In [26]:
def get_most_similars(document_vectors, query_vector, similarity_func, k=10):
    tops = list()
    for i, vec in enumerate(document_vectors):
        similarity = similarity_func(query_vector, vec)
        if not np.isnan(similarity):
          tops.append([similarity, all_data[i]])
    tops = np.array(tops)
    args = np.argsort(tops[:, 0])[::-1]
    return list(tops[args[:k]])

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(analyzer= lambda x: sent_pre_process(x, remove_stopwords=True))
tfidf_data = tfidf.fit_transform(all_data)
tfidf_data = tfidf_data.toarray()

In [28]:
def tfidf_retrieval(str, k=10):
  tfidf_str = tfidf.transform([str]).toarray()[0]
  return pd.DataFrame(get_most_similars(tfidf_data, tfidf_str, get_cosine, k))

In [29]:
tfidf_retrieval('دلم خانه‌ی مهر یار است و بس', k=10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in double_scalars
  


,0,1
0,0.6832235115151075,دلم خانهٔ مهر یار است و بس\nاز آن می‌نگنجد در ...
1,0.5413772731987289,گفت: ای یار
2,0.5413772731987289,گفت: ای یاران
3,0.3613019833857258,گفت: ای یار عزیز
4,0.310908529569272,خرابت کند شاهد خانه کن\nبرو خانه آباد گردان به زن
5,0.30932197746696155,یکی از لوازم صحبت آن است که خانه بپردازی یا با...
6,0.2855158030134369,رهی رو که بینی طریق رجا\nتو و مهر شمع از کجا ت...
7,0.2754478022726963,بر سر لوح او نبشته به زر\nجور استاد به ز مهر پدر
8,0.27440282720818304,منجمی به خانه در آمد
9,0.26910487285512996,صاحب دعوت گفت: ای یار


# Transformer-based Model

For this part, we create a model based on [Pars-BERT](https://github.com/hooshvare/parsbert), a pretrained transformer model based on BERT.

In [30]:
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 79 kB 6.8 MB/s 
     |████████████████████████████████| 4.2 MB 20.3 MB/s 
     |████████████████████████████████| 1.2 MB 36.5 MB/s 
     |████████████████████████████████| 86 kB 5.7 MB/s 
     |████████████████████████████████| 596 kB 50.9 MB/s 
     |████████████████████████████████| 6.6 MB 39.4 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=9069d22986763da542242c8b8e6a3b36741ce69db71712455c670d5a2cbc7139
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [34]:
import sentence_transformers as st
import itertools

In [35]:
class my_transformer:
    def __init__(self, document):
        self.data = document
        self.x = st.models.Transformer('HooshvareLab/bert-fa-zwnj-base')
        self.model = st.SentenceTransformer(modules=[self.x, st.models.Pooling(self.x.get_word_embedding_dimension())])
        self.processed_data = pre_process(self.data, remove_stopwords=True, put_index=True)
        self.processed_data_index = [(' '.join(x[0]), x[1]) for x in self.processed_data]
        self.processed_data = [' '.join(x[0]) for x in self.processed_data]
        self.vectors = self.model.encode(self.processed_data)
    
    def search(self, query, k=10):
        pre_processed_query = sent_pre_process(query, remove_stopwords=True)
        joint_processed_query = ' '.join(pre_processed_query)
        tops = list()
        query_encode = self.model.encode(joint_processed_query)
        sims = st.util.cos_sim(query_encode, self.vectors).numpy()[0]
        indexes = np.argsort(sims)[::-1][:k]
        tops = list()
        for ind in indexes:
            tops.append((sims[ind], self.data[self.processed_data_index[ind][1]]))
        return tops

In [36]:
transformer_model = my_transformer(all_data)

Downloading:   0%|          | 0.00/565 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/452M [00:00<?, ?B/s]

Some weights of the model checkpoint at HooshvareLab/bert-fa-zwnj-base were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at HooshvareLab/bert-fa-zwnj-base and are newly initialized: ['bert.pooler.dense.weight', 'bert.p

Downloading:   0%|          | 0.00/292 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/416k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/134 [00:00<?, ?B/s]

In [37]:
transformer_model.search('به نام خداوند جان آفرین')

[(0.78659546, 'زهی بندگان را خداوندگار\nخداوند را بندهٔ حق گزار'),
 (0.7836034, 'گناه آید از بندهٔ خاکسار\nبه امید عفو خداوندگار'),
 (0.7790897, 'پس ای مرد پوینده بر راه راست\nتو را نیست منت، خداوند راست'),
 (0.77556974, 'چو باز آمد از عیش و شادی به جای\nمرا دید و گفت ای خداوند رای'),
 (0.7660403, 'پدر بعد از آن، روزگاری شمرد\nبه جان آفرین جان شیرین سپرد'),
 (0.75382686, 'پس آن را که شخصم ز خاک آفرید\nبه قدرت در او جان پاک آفرید'),
 (0.74856436,
  'به نام خداوندِ جان\u200cآفرین\nحکیمِ سخن\u200cدرزبان\u200cآفرین'),
 (0.74346936, 'خداوند از آن بنده خرسند نیست\nکه راضی به قسم خداوند نیست'),
 (0.733925, 'تو را نیست منت ز روی قیاس\nخداوند را من و فضل و سپاس'),
 (0.73277295, 'خداوند بخشندهٔ دستگیر\nکریم خطابخش پوزش\u200cپذیر')]

# Word Embedding

We'll use Word2Vec as our model for for Word Embedding:

In [38]:
from gensim.models import Word2Vec

In [39]:
def get_all_words(document):
    all_words = {}
    for line in pre_process(document['poem'], remove_stopwords=True):
        for w in line:
            if w not in all_words:
                all_words[w] = 0
            all_words[w] += 1
    return all_words

In [40]:
print(pre_process(boostan_data['poem']))

[['به', 'نام', 'خداوند', 'جان\u200cآفرین', 'حکیم', 'سخن\u200cدرزبان\u200cآفرین'], ['خداوند', 'بخشنده', 'دستگیر', 'کریم', 'خطابخش', 'پوزش\u200cپذیر'], ['عزیزی', 'که', 'هر', 'که', 'از', 'درش', 'سر', 'بتافت', 'به', 'هر', 'در', 'که', 'شد', 'هیچ', 'عزت', 'نیافت'], ['سر', 'پادشاهان', 'گردن\u200cفراز', 'به', 'درگاه', 'او', 'بر', 'زمین', 'نیاز'], ['نه', 'گردن\u200cکشان', 'را', 'بگیرد', 'به', 'فور', 'نه', 'عذرآوران', 'را', 'براند', 'به', 'جور'], ['وگر', 'خشم', 'گیرد', 'از', 'کردار', 'زشت', 'چون', 'بازآمدی', 'ماجرا', 'درنوشت'], ['اگر', 'با', 'پدر', 'جنگ', 'جوید', 'کسی', 'پدر', 'بی\u200cگمان', 'خشم', 'گیرد', 'بسی'], ['وگر', 'خویش', 'راضی', 'نباشد', 'از', 'خویش', 'چون', 'بیگانگانش', 'براند', 'از', 'پیش'], ['وگر', 'بنده', 'چابک', 'نباشد', 'به', 'کار', 'عزیزش', 'ندارد', 'خداوندگار'], ['وگر', 'بر', 'رفیقان', 'نباشی', 'شفیق', 'به', 'فرسنگ', 'بگریزد', 'از', 'تو', 'رفیق'], ['وگر', 'ترک', 'خدمت', 'کند', 'لشکری', 'شود', 'شاه', 'لشکرکش', 'از', 'او', 'بری'], ['ولیکن', 'خداوند', 'بالا', 'و', 'پست', 'به', 'عص

In [41]:
import itertools

In [42]:
from sklearn.feature_extraction.text import TfidfTransformer

idf = TfidfTransformer(smooth_idf=True,use_idf=True) 
idf.fit(tf_data);

In [43]:
idf_data = idf.idf_
idf_data.shape
idf_dataframe = pd.Series(idf_data, index=tf.get_feature_names())
idf_dataframe.head()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


آب        5.370972
آبت       8.691200
آبدان     9.096665
آبروی     7.391917
آبرویش    8.691200
dtype: float64

In [44]:
class my_word2vec:
    def __init__(self, document):
        self.data = document
        # all_words = get_all_words(document)
        x = pre_process(document, remove_stopwords=True)
        self.model = Word2Vec(x)

    def get_most_similar_word_to_word(self, word):
        return self.model.wv.most_similar(word)

    def get_similarity(self, query, line):
        pre_processed_query = sent_pre_process(query, remove_stopwords=True)
        pre_processed_line = sent_pre_process(line, remove_stopwords=True)
        query_vector = self.word_summation(pre_processed_query)
        line_vector = self.word_summation(pre_processed_line)
        return get_cosine(query_vector, line_vector)

    def word_summation(self, sent):
        vector = np.zeros(self.model.wv.vector_size, dtype=np.float64)
        weight_sum = 0
        for word in sent:
          try:
            weight = idf_dataframe[word]
            vector += self.model.wv[word] * weight
            weight_sum += weight
          except:
            pass
        return vector / weight_sum

    def search(self, sentence, k=10):
        tops = list()
        for line in self.data:
            similarity = self.get_similarity(sentence, line)
            if not np.isnan(similarity):
              tops.append([similarity, line])
        tops = np.array(tops)
        args = np.argsort(tops[:, 0])[::-1]
        return list(tops[args[:k]])

    def get_vec(self, doc):
        return self.model.infer_vector(pre_process(doc)[0])

In [45]:
model = my_word2vec(all_data)

In [46]:
beit = 'به نام خداوند جان و خرد'

In [47]:
for x in model.search(beit):
    print(x[0], x[1])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in true_divide


0.9987198489631657 گفت: جان پدر
0.9986101365529694 به جان گفت باید نفس بر نفس
که شکرش نه کار زبان است و بس
0.998549466916384 بسی بر نیامد که بنی عمّ سلطان به منازعت خاستند و ملک پدر خواستند، قومی که از دست تطاول او به جان آمده بودند و پریشان شده بر ایشان گرد آمدند و تقویت کردند تا ملک از تصرف این به در رفت و بر آنان مقرر شد
0.9985299369191578 چو بینی دعاگوی دولت هزار
خداوند را شکر نعمت گزار
0.9984942260453006 چو بینی که لشکر ز هم دست داد
به تنها مده جان شیرین به باد
0.9984659248548472 اگر به هر سر موییت صد خرد باشد
خرد به کار نیاید چو بخت بد باشد
0.9984615857289062 گفت: ای خداوند روی زمین
0.9984615857289062 گفت: ای خداوند روی زمین
0.9984558010214365 غافلی را شنیدم که خانه رعیت خراب کردی تا خزانه سلطان آباد کند بی خبر از قول حکیمان که گفته‌اند هر که خدای را عزّ و جلّ بیازارد تا دل خلقی به دست آرد خداوند تعالی همان خلق را بر او گمارد تا دمار از روزگارش بر آرد
0.9984483521384041 بیچاره در آن حالت نومیدی ملک را دشنام دادن گرفت و سقط گفتن، که گفته‌اند هر که دست از جان بشوید، هر چه در دل دار

# MMR and Evaluating Model

In [48]:
dataset_queries = pd.read_csv('HW3-Dataset.csv', names=['query', 'user1', 'user2', 'user3'])
queries = []

In [49]:
for row in dataset_queries['query'].dropna():
    if row.startswith('q'):
        queries.append(row.split('"')[-2])

xlsx_extract, creates a xlxx file for each of four methods, containing 10 responses for each of 10 queries. We will use these files to label our retreived documents.


In [50]:
def xlsx_extract(output, response_dict):
    dataframe_list = []
    i = 1
    for key, value in response_dict.items():
        dataframe_list.append(f'q{i} = "{key}"')
        for sent in value:
            dataframe_list.append(sent)
        i += 1

    df = pd.DataFrame({'query/response': dataframe_list,
                'user1': np.nan,
                'user2': np.nan,
                'user3': np.nan})
    df.to_excel(output, index=False)  

Extracting responses for each of four models:

In [51]:
# boolean_retrieval extract xlsx
response_dict = {}

for query in queries:
    response = boolean_retrieval(query)
    response_dict[query] = []
    for number, sent in response:
        response_dict[query].append(sent)

xlsx_extract('boolean.xlsx', response_dict)

In [52]:
# tfidf_retrieval extract xlsx
response_dict = {}

for query in queries:
    response = tfidf_retrieval(query)
    response_dict[query] = []
    for sent in response[1]:
        response_dict[query].append(sent)

xlsx_extract('tfidf.xlsx', response_dict)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in double_scalars
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in double_scalars
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in double_scalars
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in double_scalars
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in double_scalars
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in double_scalars
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in double_scalars
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in do

In [53]:
queries

['به نام خداوند جان و خرد\nکز این برتر اندیشه بر نگذرد',
 'صورتی بنشینده گشتی ترجمان\nبی\u200cخبر از گفت خود چون طوطیان',
 'تو نیکو روش باش تا بد سگال\nنیابد به نقص تو گفتن مجال',
 'دلم خانهٔ مهر یار است و بس\nاز آن می\u200cنگنجد در او کین کس',
 'عجب ناید از سیرت بخردان\nکه نیکی کنند از کرم با بدان',
 'شب پره گر وصل آفتاب نخواهد\nرونق بازار آفتاب نکاهد',
 'هر که بی او به سر نشاید برد\nگر جفایی کند بباید برد',
 'صبر قفا خورد و به راهی گریخت\nعقل بلا دید و به کنجی نشست',
 'هر که بیفتاد به تیرت نخاست\nوان که درآمد به کمندت نجست\nما به تو یک باره مقید شدیم\nمرغ به دام آمد و ماهی به شست\nصبر قفا خورد و به راهی گریخت\nعقل بلا دید و به کنجی نشست\nبار مذلت بتوانم کشید\nعهد محبت نتوانم شکست\nوین رمقی نیز که هست از وجود\nپیش وجودت نتوان گفت هست',
 'دلم خانهٔ مهر یار است و بس\nاز آن می\u200cنگنجد در او کین کس',
 'ببرد از من قرار و طاقت و هوش\nبت سنگین دل سیمین بناگوش\nنگاری چابکی شنگی کلهدار\nظریفی مه وشی ترکی قباپوش\nز تاب آتش سودای عشقش\nبه سان دیگ دایم می\u200cزنم جوش\nچو پیراهن شوم آسوده خاطر

In [54]:
# word2vec extract xlsx
response_dict = {}
model = my_word2vec(all_data)

for query in queries:
    response = model.search(query)
    response_dict[query] = []
    for sent in response:
        response_dict[query].append(sent[1])

xlsx_extract('word2vec.xlsx', response_dict)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in true_divide


In [55]:
# transformer extract xlsx
response_dict = {}
model = my_transformer(all_data)

for query in queries:
    response = model.search(query)
    response_dict[query] = []
    for sent in response:
        response_dict[query].append(sent[1])

xlsx_extract('transformer.xlsx', response_dict)

Some weights of the model checkpoint at HooshvareLab/bert-fa-zwnj-base were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at HooshvareLab/bert-fa-zwnj-base and are newly initialized: ['bert.pooler.dense.weight', 'bert.p

Computing MRRs:

In [93]:
# MRR

methods = ['boolean', 'tfidf', 'word2vec', 'transformer']

for method in methods:
  df = pd.read_csv(f'labeled-output/{method}-labeled.csv', names=['query', 'user1', 'user2', 'user3'], encoding='utf-8')
  user1_vector = np.array(df['user1'])[1:]
  user2_vector = np.array(df['user2'])[1:]
  user3_vector = np.array(df['user3'])[1:]
  score_vector = np.concatenate([user1_vector.reshape([10, 11])[:, 1:],
                    user2_vector.reshape([10, 11])[:, 1:],
                    user3_vector.reshape([10, 11])[:, 1:]])

  score_list = []

  for score in score_vector:
      for i, x in enumerate(score):
          if x == '1':
              score_list.append(1/(i + 1))
              break
          elif i == 9:
              score_list.append(0)

  MRR_score = sum(score_list) / len(score_list)

  print(f'MRR score for method {method} is: {MRR_score:.3f}')

MRR score for method boolean is: 0.667
MRR score for method tfidf is: 0.702
MRR score for method word2vec is: 0.725
MRR score for method transformer is: 0.578
